In [47]:
import numpy as np

import sympy as sm
from scipy.interpolate import interp1d

global eps0, e, p, NE0, E0, F0, U1, U2, mue, mui, alpha, ne_0, N, M, r, hr, hz, R1, R2, R3, R4, R5, D
eps0 = (8.85e-12)
e = (1.6e-19)
p = 1e5
NE0 = 1e18
E0 = 5e3
F0 = 1500
U1 = -100
U2 = 0
mue = 1
mui = 1
alpha = 1
ne_0 = 1e18
k = 1.380649e-23
T = 300

N = 10
M = N

ne = np.zeros((N,M))
De = np.zeros((N,M))

#Di = 
#Dm = 

L = 1
R = 1
r1 = R
hr = (1/(N-0.5))
hz = (L/(M-1))
tEnd = 1
tau = 1
 
r = sm.zeros(1,N)
for i in range(N):
    r[i] = (hr/2 + (R-hr/2)/(N-1)*(i)) #r = np.linspace(hr/2,R,N)

z = sm.zeros(1,M)
for i in range(M):
    z[i] = (0 + L/(M-1)*(i)) 
#z = np.linspace(0,L,M)

FF = sm.zeros(N,M)
Net = np.zeros((N,M))
for j in range(0,M):
    for i in range(0,N):
        FF[i,j] = (-50*z[j]**2)
        Net[i,j] = p/(k*T)
        ne[i][j] = ne_0*(1-(r[i]/R)**2)*(z[j]*(L-z[j]))
ni = ne.copy()
nm = ne.copy()

for i in range(N):
    FF[i,0] = 0 #низ
for i in range(N):
    FF[i,M-1] = -50



In [48]:
Di = np.zeros((N,2))
R1 = np.zeros((N,2))
patternD = 'Diffusion coefficient'
patternR = 'C1    Ar    Effective (momentum)'
with open('output.dat', 'r') as file:
    for index, line in enumerate(file):
        if patternD in line:
            indexD = index
        elif patternR in line:
            indexR = index
            break
file = open('output.dat', 'r')
lines = file.readlines()
for i in range(N):
    Di[i] = [float(lines[indexD+1+i][:5]),float(lines[indexD+1+i][-11:-1])]
    R1[i] = [float(lines[indexR+2+i][:5]),float(lines[indexR+2+i][-11:-1])]
file.close()

R1_linear = interp1d(list(i[0] for i in R1), list(i[1] for i in R1), kind = 'linear')
Di_linear = interp1d(list(i[0] for i in Di), list(i[1] for i in Di), kind = 'linear')

In [49]:

R2 = 6.2e-10
R3 = 2e-7*np.exp(-6.2/((T*k)/e))
R4 = 2.7e-13*((T*k)/e)**(-3/4)
R5 = 8.75e-39*((T*k)/e)**(-9/2)

In [50]:
def decLU3(c, a, b):
    # print("Это LU3")
    # print("c=", c,"a=", a,"b=", b)
    """
    Input of tridiagonal matrix A:
        a[i] = A[i,i],
        b[i] = A[i,i+1],
        c[i] = A[i,i-1]
    Returns thr decomposition LU for tridiagonal matrix:
        d[i] = L[i,i]
        u[i] = U[i,i+1]
        l[i] = L[i,i-1]
    """
    n = len(a)
    d = np.copy(a)
    u = np.copy(b)
    l = np.copy(c)
    for i in range(1, n):
        al = l[i] / d[i-1]
        d[i] = d[i] - al*u[i-1]
        l[i] = al
    return d, u, l
def solveLU3(c, a, b, f):
    """
    Solve the linear system Ax = b with tridiagonal matrix:
        a[i] = A[i,i]
        b[i] = A[i,i+1],
        c[i] = A[i,i-1]
    """
    n = len(a)
    # LU decomposition
    d, u, l = decLU3(c, a, b)
    x = np.copy(f)
    # forward substitution process
    for i in range(1, n):
        x[i] = x[i] - l[i]*x[i-1]
    # back substitution process
    x[n-1] = x[n-1] / d[n-1]
    for i in range(n-2,-1,-1):
        x[i] = (x[i] - u[i]*x[i+1]) / d[i]
    return x

In [51]:
def parabolic2D(r, z, f, y_0, znak, mu, phi, D, tau):
    """
    znak -- знак заряда частицы (+1 для иона, -1 для электрона)
    Numerical Solution of the Dirichlet problem
    for two-dimentional parabolic equation.
    Use additive difference scheme of alternating directions.
    r: 1D NumPy array that contains the x-coordinates of the grid points.
    z: 1D NumPy array that contains the y-coordinates of the grid points.
    f: 2D NumPy array that contains the values of the right-hand side function at each grid point.
    y_0: 2D NumPy array that contains the initial guess for the solution.
    znak: scalar that represents the sign of the charge of the particles (1 for an ion, -1 for an electron).
    mu: scalar that represents the magnetic permeability of the medium.
    phi: 2D NumPy array that contains the values of the electric potential at each grid point.
    D: 2D NumPy array that contains the values of the diffusion coefficient in the x and y directions, respectively.
    tau: scalar that represents the time step.
    """
    # print('PARABOLIC2d')
    # print('Нач. приближение \n', y_0)
    n1 = len(r)
    n2 = len(z)
    h1 = r[1]-r[0]
    h2 = z[1]-z[0]
    
    a1 = np.zeros((n1), 'float')
    b1 = np.ones((n1), 'float')
    c1 = np.zeros((n1), 'float')
    q1 = np.zeros((n1), 'float')
    a2 = np.zeros((n2), 'float')
    b2 = np.ones((n2), 'float')
    c2 = np.zeros((n2), 'float')
    q2 = np.zeros((n2), 'float')
    
    D_r = np.ones((n1, n2), 'float')
    E_r = np.ones((n1, n2), 'float')*10.
    alpha_r = np.ones((n1, n2), 'float')
    I_0_r = np.ones((n1, n2), 'float')
    
    D_z = np.ones((n1, n2), 'float')
    E_z = np.ones((n1, n2), 'float')*10.
    alpha_z = np.ones((n1, n2), 'float')
    I_0_z = np.ones((n1, n2), 'float')
    
    y0 = y_0
    y = np.copy(y_0)
    
    # while t0 < tEnd - 0.001*tau:
    # x1 direction
    # вычисляем коэффициенты в промежуточных узлах
    for j in range(0,n2-1):
        D_r[0,j] = (D[0,j]+D[1,j])/2
        E_r[0,j] = (phi[1,j]-phi[0,j])/h1
        alpha_1 = -znak*mu*E_r[0,j]*h1/D_r[0,j]
        alpha_r[0,j] = alpha_1
        if np.isclose(alpha_1, 0.0, atol=1e-8):
            I_0_r[0,j] = 1
        else:
            I_0_r[0,j] = (np.exp(alpha_r[0,j])-1)/alpha_r[0,j]
        
        for i in range(0,n1-1):
            D_r[i,j] = (D[i,j]+D[i+1,j])/2
            E_r[i,j] = (phi[i,j]-phi[i+1,j])/h1
            alpha_1 = -znak*mu*E_r[i,j]*h1/D_r[i,j]
            alpha_r[i,j] = alpha_1
            if np.isclose(alpha_1, 0, atol=1e-8):
                I_0_r[i,j] = 1
            else:
                I_0_r[i,j] = (np.exp(alpha_r[i,j])-1)/alpha_r[i,j]
            
    for i in range(0,n1-1):
        D_z[i,0] = (D[i,0]+D[i,1])/2
        E_z[i,0] = (phi[i,1]-phi[i,0])/h2
        alpha_1 = -znak*mu*E_z[i,0]*h2/D_z[i,0]
        alpha_z[i,0] = alpha_1
        if np.isclose(alpha_1, 0.0, atol=1e-8):
            I_0_z[i,0] = 1.
        else:
            I_0_z[i,0] = (np.exp(alpha_z[i,0])-1)/alpha_z[i,0]
        
        for j in range(0,n2-1):
            D_z[i,j] = (D[i,j]+D[i,j+1])/2
            E_z[i,j] = (phi[i,j]-phi[i,j+1])/h2
            alpha_1 = -znak*mu*E_z[i,j]*h2/D_z[i,j]
            alpha_z[i,j] = alpha_1
            if np.isclose(alpha_1, 0.0, atol=1e-8):
                I_0_z[i,j] = 1.
            else:
                I_0_z[i,j] = (np.exp(alpha_z[i,j])-1)/alpha_z[i,j]                

    for j in range(1,n2-1):
        b1[0] = (2/h1**2)*D_r[0,j]/I_0_r[0,j] + 2./tau
        c1[0] = -(2/h1**2)*D_r[0,j]/I_0_r[0,j]
        q1[0] = (0.5*f[0,j]) + 2*y[0,j]/tau \
            + (D_z[0,j-1]/(I_0_z[0,j-1])*y[0,j-1] \
                - (D_z[0,j-1]/(I_0_z[0,j-1])*np.exp(alpha_z[0,j-1]) \
                  + D_z[0,j]/(I_0_z[0,j]))*y[0,j] \
                    + D_z[0,j]/(I_0_z[0,j])*np.exp(alpha_z[0,j])*y[0,j+1])/h2**2
        
        for i in range(1,n1-1):
            a1[i] = -((r[i]-h1/2))*D_r[i-1,j]/(r[i]*h1**2*I_0_r[i-1,j])
            b1[i] = ((r[i]-h1/2))*D_r[i-1,j]/(r[i]*h1**2*I_0_r[i-1,j])\
                *np.exp(alpha_r[i-1,j]) \
                + (r[i]+h1/2)/r[i]*D_r[i,j]/(h1**2*I_0_r[i,j]) + 2. / tau
            c1[i] = -(r[i]+h1/2)/r[i]*D_r[i,j]/(h1**2*I_0_r[i,j]) \
                *np.exp(alpha_r[i,j])
            q1[i] = f[i,j]/2. + 2.*y[i,j]/tau \
                + (D_z[i,j-1]/(h2*I_0_z[i,j-1])*y[i,j-1] \
                   - (D_z[i,j-1]/(h2*I_0_z[i,j-1])*np.exp(alpha_z[i,j-1]) \
                      + D_z[i,j]/(h2*I_0_z[i,j]))*y[i,j] \
                       + D_z[i,j]/(h2*I_0_z[i,j])*np.exp(alpha_z[i,j])*y[i,j+1])/h2   
        y0[:,j] = solveLU3(a1, b1, c1, q1)
        
    for j in range(1,n2-1):
        a2[j] = -D_z[0,j-1]/(h2**2*I_0_z[0,j-1])
        b2[j] = -a2[j]*np.exp(alpha_z[0,j-1])+D_z[0,j]/(h2**2*I_0_z[0,j]) \
            + 2./tau
        c2[j] = -D_z[0,j]/(h2**2*I_0_z[0,j])*np.exp(alpha_z[0,j])
        q2[j] = f[0, j]/2. + 2.*y0[0,j]/tau \
            - (((r[0]+h1/2)/r[0]*D_r[0,j]/(I_0_r[0,j]))*y0[0,j]\
                   - (r[0]+h1/2)/r[0]*D_r[0,j]/(I_0_r[0,j])\
                       *np.exp(alpha_r[0,j])*y0[1,j])/h1**2    
    y[0,:] = solveLU3(a2, b2, c2, q2)
    
    # x2 direction
    for i in range(1,n1-1):
        for j in range(1,n2-1):
            a2[j] = -D_z[i,j-1]/(h2**2*I_0_z[i,j-1])
            b2[j] = -a2[j]*np.exp(alpha_z[i,j-1])+D_z[i,j]/(h2**2*I_0_z[i,j]) \
                + 2. / tau
            c2[j] = -D_z[i,j]/(h2**2*I_0_z[i,j])*np.exp(alpha_z[i,j])
            q2[j] = f[i, j]/2. + 2.*y0[i,j]/tau\
                - (-(r[i]-h1/2)/D_r[i-1,j]/(I_0_r[i-1,j])*y0[i-1,j]\
                   + ((r[i]-h1/2)/D_r[i-1,j]/(I_0_r[i-1,j])\
                      *np.exp(alpha_r[i-1,j])\
                      + (r[i]+h1/2)/D_r[i,j]/(I_0_r[i,j]))*y0[i,j]\
                       - (r[i]+h1/2)/D_r[i,j]/(I_0_r[i,j])\
                           *np.exp(alpha_r[i,j])*y0[i+1,j])/r[i]/h1**2      
        y[i,:] = solveLU3(a2, b2, c2, q2)
    return y

In [52]:
def EEEE(phi):
    r = np.linspace(hr/2,R,N)
    n1 = len(r)
    n2 = len(z)
    h1 = r[1]-r[0]
    h2 = z[1]-z[0]
    E_r = np.ones((n1, n2), 'float')*10.
    E_z = np.ones((n1, n2), 'float')*10.
    for j in range(0,n2-1):
        E_r[0,j] = (phi[1,j]-phi[0,j])/h1
        for i in range(0,n1-1):
            E_r[i,j] = (phi[i,j]-phi[i+1,j])/h1
    for i in range(0,n1-1):
        E_z[i,0] = (phi[i,1]-phi[i,0])/h2
        for j in range(0,n2-1):
            E_z[i,j] = (phi[i,j]-phi[i,j+1])/h2
    return 

In [53]:
def Rg_pr(FF):
    f = np.zeros((N,M))
    for i in range(N):
        for j in range(M):
            variable_phi=FF[i,j]/Net[i,j]
            if variable_phi < 1: variable_phi = 1
            f[i,j] = R1_linear(float(variable_phi))*ne[i,j]*Net[i,j] + R2*nm[i,j]**2 + R3*nm[i,j]*ne[i,j] - R4*ne[i,j]*ni[i,j] - R5*ne[i,j]**2*ni[i,j]

In [56]:
def fsolve(FF,ne,ni,omega_relax = 1.8):
    FF1 = FF.copy()
    f = sm.ones(N,M) * (100)
    d = sm.ones(1,N*M) #формируем диагональ матрицы СЛАУ
    l1 = sm.zeros(1,N*M) 
    l2 = sm.zeros(1,N*M)
    for j in range(1,M-1):
        for i in range(1,N-1):
            k = i+j*N
            d[k] = 2/hr**2 +2/hz**2
            l1[k] = -(r[i]-hr/2)/(r[i]*hr**2)
            l2[k] = -1/hz**2
        l1[int((j+1)*N-1)] = -1/hr**2

    tol = (1e-2)
    rn_finish = (1e32)

    kiter = 0
    N_iter = 5

    for p in range(N_iter):
        norm_dif = 0
        rn = 0
        print('rn = ',float(rn))

        print("Расчет")
      
        resid = sm.zeros(1,N*M)
        corrector = sm.zeros(1,N*M)


        for j in range(1,M-1):
            # Граничные условия второго рода слева  
            rr = (FF[0,j]*(2/hr**2) - FF[1,j]*(2/hr**2))\
                - 1/hz**2 * FF[0,j-1] + 2/hz**2 * FF[0,j] - 1/hz**2 * FF[0,j+1]\
                - f[0,j]
            resid[j*N] = rr
            rn = rn+rr**2
            print('rr невязка левые граница', rr )
            FF1[0,j] = FF[0,j] - (omega_relax*rr)/d[j*N]
            norm_dif = max(norm_dif,abs((omega_relax*rr)/(d[j*N])))
            
            # Обходим внутренние узлы
            for i in range(1,N-1):
                rr = -(FF[i-1,j]*(r[i]- hr/2) -2*FF[i,j]*r[i] + FF[i+1,j]*(r[i]+hr/2)) / (hr**2 * r[i]) -(FF[i,j-1] - 2*FF[i,j] + FF[i,j+1])/hz**2 - f[i,j]
                corrector[i+j*N] = rr - omega_relax*(l1[i+j*N-1]*corrector[i+j*N -1] 
                                                     - l2[i+(j-1)*N]*corrector[i+(j-1)*N])/d[i+j*N]
              
                resid[i+j*N] = rr
                rn = rn+rr**2
                print ('i = ',i,'j = ', j, 'rr = ',float(rr),'rn = ',float(rn))
                FF1[i,j] = FF[i,j] - omega_relax*corrector[i+j*N]
                norm_dif = max(norm_dif,abs(omega_relax*corrector[i+j*N]))

            # справа
            rr = (FF[N-1,j] - FF[N-2,j])*2 / hr**2 - f[N-1,j]\
                - 1/hz**2 * (FF[N-1,j-1] - 2*FF[N-1,j] + FF[N-1,j+1])
            corrector[(j+1)*N-1] = rr - omega_relax*(l1[(j+1)*N-2]*corrector[(j+1)*N-2] 
                                                    - l2[j*N-1]*corrector[j*N-1])/d[(j+1)*N-1]
            print('невязка правой границы',float(rr),'\n')
            resid[(j+1)*N-1] = rr
            rn = rn+rr**2
            FF1[N-1,j] = FF[N-1,j] - (omega_relax*corrector[(j+1)*N-1])
            norm_dif = max(norm_dif,abs(omega_relax*corrector[(j+1)*N-1]))
      
        ddd=np.max(np.abs(FF-FF1))
        try:
            err_c = norm_dif/ddd
        except:
            err_c = 0
        FF=FF1.copy()
        rn = rn*hr*hz
        kiter = p
        rn_finish = rn
        #print("итерация номер ", k, " невязка = ", rn," погрешность = ", err_c)
        if rn_finish < tol**2:
            break
      
    print("число итераций =  ", kiter+1)
    print("невязка приближенного решения",rn)
    print("относительная разность, norm_dif = ", norm_dif)
    return FF,rn

In [55]:
print('решение')
t = 0
while t<= tEnd:
    
    FF,nv = fsolve(FF,ne,ni)
    ne = parabolic2D(r,z,Rg_pr(FF),ne,-1,mue,FF,Di,tau)
    ni = parabolic2D(r,z,Rg_pr(FF),ni,1,mui,FF,Di,tau)
    #nm = parabolic2D(r,z,_,nm,0,mui,FF,Di,tau)
    t = tau+t
    

print((FF.transpose()))


решение
rn =  0.0
Расчет
rr невязка левые граница 0
i =  1 j =  1 rr =  0.0 rn =  0.0
i =  2 j =  1 rr =  0.0 rn =  0.0
i =  3 j =  1 rr =  0.0 rn =  0.0
i =  4 j =  1 rr =  1.4210854715202004e-14 rn =  2.0194839173657902e-28
i =  5 j =  1 rr =  1.4210854715202004e-14 rn =  4.0389678347315804e-28
i =  6 j =  1 rr =  1.4210854715202004e-14 rn =  6.058451752097371e-28
i =  7 j =  1 rr =  0.0 rn =  6.058451752097371e-28
i =  8 j =  1 rr =  -1.4210854715202004e-14 rn =  8.077935669463161e-28
невязка правой границы 0.0 

rr невязка левые граница 0
i =  1 j =  2 rr =  1.4210854715202004e-14 rn =  1.0097419586828951e-27
i =  2 j =  2 rr =  1.4210854715202004e-14 rn =  1.2116903504194741e-27
i =  3 j =  2 rr =  1.4210854715202004e-14 rn =  1.4136387421560532e-27
i =  4 j =  2 rr =  5.684341886080802e-14 rn =  4.6448130099413175e-27
i =  5 j =  2 rr =  4.263256414560601e-14 rn =  6.462348535570529e-27
i =  6 j =  2 rr =  4.263256414560601e-14 rn =  8.27988406119974e-27
i =  7 j =  2 rr =  1.421

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
Matrix=FF.copy()
for i in range(N): 
    for j in range(N): 
        print(float(FF[i,j]), end = ' ')
    print()
for i in range(N): 
    for j in range(N): 
        print(float(Matrix[i,j]), end = ' ')
    print()

print()
FFF = sm.zeros(N,M)
for j in range(0,M):
    for i in range(0,N):
        FFF[i,j] = (-50*z[j]**2)
for i in range(N):
    FFF[i,0] = 0 #низ
for i in range(N):
    FFF[i,M-1] = -50
for i in range(N): 
    for j in range(N): 
        print(float(FFF[i,j]), end = ' ')
    print()

print()
for i in range(N): 
    for j in range(N): 
        print(float(Matrix[i,j]-FFF[i,j]), end = ' ')
    print()
